In [1]:
import pandas as pd
import numpy as np
from collections import *
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import treebank
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from pymagnitude import *
import time
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import random
import mxnet as mx
from mxnet import gluon
import gluonnlp as nlp
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import bert
from bert import *
from tensorflow import keras
import os
import re

W0503 17:36:27.294411 140599383648064 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')

In [ ]:
data = pd.read_csv('data/questions.csv')
data = data
path = 'data/GoogleNews-vectors-negative300.magnitude'
vectors = Magnitude(path)
tok = RegexpTokenizer('\w+')
data.question1 = pd.Series([tok.tokenize(str(s)) for s in data.question1])
data.question2 = pd.Series([tok.tokenize(str(s)) for s in data.question2])

In [ ]:
# helper functions and constants
def sum_vec(vecs):
    res = np.sum(vecs, axis=0)
    if type(res) is np.float64:
        return np.array([1e-32] * vectors.dim)
    else:
        return res
    
def rm_stop(words):
    stop_words = nltk.corpus.stopwords.words('english')
    return [w for w in words if not w in stop_words]

def train_test(col1 ,col2):
    sep = int(len(data)*.75)
    ret = {}
    ret['train_pnts'] = data.as_matrix([col1, col2])[:sep]
    ret['train_pnts'] = [[cosine(x[0], x[1])] for x in ret['train_pnts']]
    ret['train_tgts'] = data.as_matrix(["is_duplicate"])[:sep]
    ret['test_pnts']  = data.as_matrix([col1, col2])[sep:]
    ret['test_pnts']  = [[cosine(x[0], x[1])] for x in ret['test_pnts']]
    ret['test_tgts']  = data.as_matrix(["is_duplicate"])[sep:]
    return ret

def to_vec(words):
    return [vectors.query(w) for w in words if w in vectors]

i_tup_num = 0
i_question1 = 3
i_question2 = 4
i_is_duplicate = 5
i_vec1 = 6
i_vec2 = 7
i_stop1 = 8
i_stop_vec1 = 9
i_stop2 = 10
i_stop_vec2 = 11

In [ ]:
t = time.time()
vec1 = pd.Series(to_vec(words) for words in data.question1)
#vec1 = pd.Series(sum_vec(words) for words in data.question1)
data.insert(len(data.columns), "vec1", vec1)
print("Finished vec1")
vec2 = pd.Series(sum_vec(words) for words in data.question2)
data.insert(len(data.columns), "vec2", vec2)
print("Finished vec2")
print(time.time() - t)
t = time.time()
stop1 = pd.Series(rm_stop(words) for words in data.question1)
data.insert(len(data.columns), "stop1", stop1)
stop_vec1 = pd.Series(sum_vec(words) for words in data.stop1)
data.insert(len(data.columns), "stop_vec1", stop_vec1)
print("Finished stop1")
stop2 = pd.Series(rm_stop(words) for words in data.question2)
data.insert(len(data.columns), "stop2", stop2)
stop_vec2 = pd.Series(sum_vec(words) for words in data.stop2)
data.insert(len(data.columns), "stop_vec2", stop_vec1)
print("Finished stop2")
print(time.time() - t)

In [ ]:
#initial data information
print("# of data points:", len(data))
print("# of word vectors:", len(vectors))
print('vector dimensions:', vectors.dim)
cnt = Counter(data.is_duplicate)
print("random baseline:", 1/len(cnt.keys()))
print("most common baseline:", max([cnt[k]/sum(cnt.values()) for k in cnt.keys()]))

In [ ]:
accuracy_score(
    [0 if nltk.translate.bleu_score.sentence_bleu(
    [data.question1[i]], 
    data.question2[i]) < .5 else 1
    for i in range(len(data))],
    data.is_duplicate)

In [ ]:
x = train_test("vec1", "vec2")
lr = LogisticRegression().fit(x['train_pnts'], x['train_tgts'])
print("normal embeddings LR accuracy:", accuracy_score(lr.predict(x['test_pnts']), x['test_tgts']))
print("normal embeddings LR coefficients:", lr.coef_)

In [ ]:
x = train_test("stop_vec1", "stop_vec2")
lr = LogisticRegression().fit(x['train_pnts'], x['train_tgts'])
print("no stop words LR accuracy:", accuracy_score(lr.predict(x['test_pnts']), x['test_tgts']))
print("no stop words LR coefficients:", lr.coef_)

In [ ]:
i = 0
for x in data.values:
    if x[i_question1] == x[i_question2]:
        print(x[i_is_duplicate], x[i_tup_num])
        print(x[i_question1])
        i += 1
print(i)

In [ ]:
hello = data.as_matrix(["vec1", "vec2"])
hello = [[cosine(x[0], x[1])] for x in hello]

In [ ]:
x = data.values
k = 0
for i in range(len(data.values)):
    if hello[i] == [0.0]:
        print(x[i][i_is_duplicate], x[i][i_tup_num])
        print(x[i][i_question1])
        print(x[i][i_question2])
        k += 1
print(k)

In [ ]:
nltk.corpus.stopwords.words('english')

In [ ]:
cosine(vectors.query("PhD"), vectors.query("Ph.D."))

In [ ]:
"WW1" in vectors

In [ ]:
"Chick-fil-A" in vectors

In [ ]:
data[299692:299692+1]

In [ ]:
from nltk.corpus import treebank

trees = treebank.parsed_sents()

In [ ]:
nltk.download('treebank')

In [ ]:
print(trees[0])

In [ ]:
x = 'dsafa'
x.hello = 1

In [ ]:
def filter(tree):
    child_nodes = [child.label() for child in tree
                   if isinstance(child, nltk.Tree)]
    return child_nodes

trees = [subtree for tree in treebank.parsed_sents() for subtree in tree.subtrees(filter)]

production_rules = defaultdict(list)
terminals = list()

for tree in trees:
    for prod in tree.productions():
        lhs = str(prod.lhs()).replace('=','-').replace('|','-')
        if prod.is_lexical():
            terminals.append(lhs)
        if prod.is_nonlexical():
            rhs = ''
            for s in prod.rhs(): 
                rhs += str(s).replace('=','-').replace('|','-') + ' '
            production_rules[lhs].append(str(rhs.strip()))
            
terminals = set(terminals)
terminals.remove("''")

production_counters = dict()

for prod in production_rules:
    production_counters[prod] = Counter(production_rules[prod])

def to_prod_rules(lhs, c):
    global terminals
    denom = sum(c.values())
    cfg_str = list()
    for rhs in c:
        rhs_prods = rhs.split()
        r = ''
        for rule in rhs_prods: 
            if rule == '' or rule == "''": continue
            r += "'" + rule + "' " if rule in terminals else rule + " "
        cfg_str.append("{} -> {} [{}]".format(lhs, r.strip(), '{:.20f}'.format(c[rhs] / denom)))
    return cfg_str

production_lhs = list(production_counters.keys())
production_lhs.remove('S')
production_lhs = ['S'] + production_lhs

prod_rules = list()

for lhs in production_lhs:
    prod_rules += to_prod_rules(lhs, production_counters[lhs])
    
prod_rules_str = '\n'.join(prod_rules)

grammar = nltk.PCFG.fromstring(prod_rules_str)

viterbi_parser = nltk.ViterbiParser(grammar)

In [ ]:
sentence = ['I', 'like', 'to', 'eat', 'tacos']

pos_sentence = nltk.pos_tag(sentence)

print(sentence)
print(pos_sentence)


for tree in viterbi_parser.parse([x[1] for x in pos_sentence]):
    print(tree)

In [ ]:
nltk.pos_tag(['I'])

In [ ]:
nltk.pos_tag(['i'])

In [ ]:
data.question1[0]

In [ ]:
bm = BertModel()

In [ ]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

In [ ]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

In [ ]:
import requests
for h in dir(requests.packages.idna):
    print(h)

In [9]:
# https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb#scrollTo=6o2a5ZIvRcJq
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

In [3]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

In [4]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [5]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [6]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [8]:
train, test = download_and_load_datasets()
train = train.sample(5000)
test = test.sample(5000)
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
tokenizer = create_tokenizer_from_hub_module()
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

OUTPUT_DIR = 'tf_out'

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

estimator.evaluate(input_fn=test_input_fn, steps=None)

pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!"
]

predictions = getPrediction(pred_sentences)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0503 16:46:14.839184 140109432092480 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 5000


I0503 16:46:15.288258 140109432092480 run_classifier.py:774] Writing example 0 of 5000


INFO:tensorflow:*** Example ***


...

I0503 16:47:20.854492 140109432092480 session_manager.py:493] Done running local_init_op.


INFO:tensorflow:Saving checkpoints for 0 into tf_out/model.ckpt.


I0503 16:47:29.267897 140109432092480 basic_session_run_hooks.py:594] Saving checkpoints for 0 into tf_out/model.ckpt.


INFO:tensorflow:loss = 0.6883311, step = 1


I0503 16:52:25.956855 140109432092480 basic_session_run_hooks.py:249] loss = 0.6883311, step = 1


In [22]:
predictions

NameError: name 'predictions' is not defined